基本上没有什么顺序可言。模型都是比较熟悉的。但因为使用PyTorch，所以很多语法都不熟悉，所以就是把觉得比较新奇的语句给摘出来。<br>
<br>
一直搞不懂的是，为什么这些类明明没有实现__call__方法，但是都可以直接调用，而且好像默认调用了其中的forward方法。是因为nn.Modul中实现了__call__方法，而其中就有调用forward这个语句吗？？？有时间应该去看看源码。

### argparse.ArgumentParser
是python的一个命令行解析包。相当于tensorflow中的FLAGS吧

In [6]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("echo") # 定义一个叫echo的参数，默认必选
# 定义了可选参数-v或--verbosity，通过解析后，其值保存在args.verbosity变量中
parser.add_argument("-v", "--verbosity", help = "increase output verbosity")
# 默认为True，不出现则为False
parser.add_argument("-v2", "--verbosity2", help = "increase output verbosity", action = "store_true")
# argparse提供了对参数类型的解析，如果类型不符合，则直接报错
parser.add_argument("-x", type = int, help = "the base")
# 可以设置可选值
parser.add_argument("-v3", "--verbosity3", type = int, choices = [0, 1, 2], help = "increase output verbosity")

args = parser.parse_args()
if args.verbosity3 == 2:
    print("th")



usage: ipykernel_launcher.py [-h] [-v VERBOSITY] [-v2] [-x X] [-v3 {0,1,2}]
                             echo
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/Users/ihuangyiran/anaconda2/envs/py3-tf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Encoder
主要确定：
- 是否bidirection
- 层的数量
- 每层节点的数量
- 使用什么层结果RNN，LSTM还是Transformer

### assert opt.rnn_size % self.num_directions == 0
使用assert断言是学习python一个非常好的习惯，python assert 断言句语格式及用法很简单。在没完善一个程序之前，我们不知道程序在哪里会出错，与其让它在运行最崩溃，不如在出现错误条件时就崩溃，这时候就需要assert断言的帮助。<br>
python assert断言是声明其布尔值必须为真的判定，如果发生异常就说明表达示为假。可以理解assert断言语句为raise-if-not，用来测试表示式，其返回值为假，就会触发异常。<br>
assert的异常参数，其实就是在断言表达式后添加字符串信息，用来解释断言并更好的知道是哪里出了问题。格式如下：<br>
assert expression [, arguments]

In [11]:
assert 1==2, "坑爹啊"

AssertionError: 坑爹啊

### self.hidden_size = opt.rnn_size // self.num_directions
opt.rnn_size: size of LSTM hidden states<br>
python 2.x里面，// 是地板除，/如果有一个数是浮点数就得到小数，如果两个都是整数也是地板除。<br>
python 3.x里面，// 是地板除，/ 不管两边是不是整数得到的都是小数。<br>
四舍五入请用: round(5/3,2)

In [10]:
a = 5 // 3 
b = 5 / 3
c = round(5/3)
d = round(5/3, 2)
a, b, c, d

(1, 1.6666666666666667, 2, 1.67)

### nn.ModuleList(Modules = None)
nn.ModuleList(<br>
&emsp;[onmt.modules.TransformerEncoder(self.hidden_size, opt)<br>
&emsp;for i in range(opt.layers)])<br>
transformerEncoder实现每层由两部分组成：self-attention层和positionwiseFeedForward（bottleLinear? + ReLu + dropout + residual）层<br>
Holds submodules in a list.<br>
ModuleList can be indexed like a regular Python list, but modules it contains are properly registered, and will be visible by all Module methods.<br>
Parameters:	modules (list, optional) – a list of modules to add<br>

### self.rnn = getattr(nn, opt.run_type)(...)
getattr(object, name[, default])<br>
Return the value of the named attribute of object. name must be a string. If the string is the name of one of the object’s attributes, the result is the value of that attribute. For example, getattr(x, 'foobar') is equivalent to x.foobar. If the named attribute does not exist, default is returned if provided, otherwise AttributeError is raised.

In [13]:
# 这样搭网络感觉牛牛哒。 另外nn.Module还是很陌生，
if self.encoder_layer = "transformer":
    self.transformer = nn.ModuleList(
        [onmt.modules.TransformerEncoder(self.hidden_size, opt)
         for i in range(opt.layers)])
else:
    self.rnn = getattr(nn, opt.rnn_type)(
        input_size, self.hidden_size, num_layers = out.layers,
        dropout = out.dropout,
        bidirectional = opt.brnn)

SyntaxError: invalid syntax (<ipython-input-13-ed73608e3478>, line 1)

### pad_attn_mask = seq_k.data.eq(onmt.Constants.PAD).unsequeeze(1).expand(mb_size, len_q, len_k)
- torch.eq(input, other, out=None) → Tensor: <br>

Computes element-wise equality. The second argument can be a number or a tensor whose shape is broadcastable with the first argument.<br>
return: a torch.ByteTensor containing a 1 at each location where the tensors are equal and a 0 at every other location<br>
- torch.unsqueeze(input, dim, out=None)<br>

Returns a new tensor with a dimension of size one inserted at the specified position. The returned tensor shares the same underlying data with this tensor.<br>
A negative dim value can be used and will correspond to dim+input.dim()+1<br>
- expand(tensor, sizes) → Tensor<br>

Returns a new view of the tensor with singleton dimensions expanded to a larger size.<br>
Tensor can be also expanded to a larger number of dimensions, and the new ones will be appended at the front.<br>
Expanding a tensor does not allocate new memory, but only creates a new view on the existing tensor where a dimension of size one is expanded to a larger size by setting the stride to 0. Any dimension of size 1 can be expanded to an arbitrary value without allocating new memory.<br>
这是一个生成mask的代码。torch.eq找出向量中的特殊单词（Constants.PAD），unsequeeze是增加一个维，该维的长度为1，expand扩展（复制）该mask的维度扩展成给定的维度。因为这个mask是给matmal(Q,K)用的，所以其维度[batch_size, seq_q, seq_k]。<br>

forward中的CHECK不明白是在干什么？？

## Decoder


### stackedCell = onmt.modules.stackedLSTM
看了stackedLSTM，实在是没有看出区别在哪里，难道nn.LISTM实现的时候，除了第一层都是不加input的吗？？只能这么理解了。没事还是多翻翻源代码吧？？？input_feeding应该是指用额外的向量来表达已经翻译够的单词。

In [15]:
# 类什么的，也是可以这么玩的。
if opt.rnn_type == "LSTM":
    stackedCell = onmt.modules.StackedLSTM
else:
    stackedCell = onmt.modules.StackedGRU
self.rnn = stackedCell(opt.layers, input_size, opt.rnn_size, opt.dropout)

NameError: name 'opt' is not defined

### ContextGateFactory
工厂模式啊，好久没见过了，是这么弄的吗？？？这里主要用来调整attention和decoder state对当前循环的影响的大小。

In [16]:
def ContextGateFacctory(type, embeddings_size, decoder_size, attention_size, output_size):
    gate_types = {'source': SourceContextGate,
                 'target': TargetContextGate,
                 'both': BothContextGate}
    assert type in gate_types, "not valid contextGate type: {0}".format(type)
    return gate_type[type](embeddings_size, decoder_size, attention_size, output_size)

### attention
- view(*args) → Tensor

Returns a new tensor with the same data but different size.<br>
The returned tensor shares the same data and must have the same number of elements, but may have a different size. A tensor must be contiguous() to be viewed.
- expand_as(tensor)<br>

Expands this tensor to the size of the specified tensor.
- contiguous() → Tensor

Returns a contiguous Tensor containing the same data as this tensor. If this tensor is contiguous, this function returns the original tensor.

### Embeddings
#### pe[:, 0::2] = torch.sin(pe[:, 0::2])
seq[start:end:step]
#### make_positional_encoding(self, dim, max_len):
可以看懂代码，但是不知道为什么要这么弄？？
#### if opt.feat_merge == 'concat': emb_sizes.extend([int(feat_dict.size() ** feat_exp) for feat_dict in feature_dicts])
为什么要弄成一样？？？不了解他的好处在哪里？？？
#### features = [lut(feat) for lut, feat in zip(self.emb_luts, feat_inputs)]
还是摆弄方法名，觉得有意思，就又写了一次。
#### emb = self.dropout(emb)
为什么在这里加dropout，其实不知道dropout一般在哪里加？？？

### RNNDecoderState(DecoderState)
#### if not isinstance(rnnsatete, tuple): self.hidden = (rnnstate,) else: self.hidden = rnnstate
对tupel不是很了解？？？所以并不理解其中的区别？？？

### make_base_model
#### class torch.nn.Sequential(*args)[source]
A sequential container. Modules will be added to it in the order they are passed in the constructor. Alternatively, an ordered dict of modules can also be passed in.

In [ ]:
# 例子
# Example of using Sequential
model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )

# Example of using Sequential with OrderedDict
model = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))

## Train.py
#### dill
dill extends python’s pickle module for serializing and de-serializing python objects to the majority of the built-in python types. Serialization is the process of converting an object to a byte stream, and the inverse of which is converting a byte stream back to on python object hierarchy.<br>
dill provides the user the same interface as the pickle module, and also includes some additional features. <br>
dill can be used to store python objects to a file, but the primary usage is to send python objects across the network as a byte stream.<br>

#### opts
opts is a simple python library which allows you to easiely parse command line arguments.

In [1]:
from opts import Parser, Command, Option, BooleanOption

parser = Parser(description=u"Our own awesome dvcs", commands={
    "add": Command(
        short_description=u"Adds a file to the repository",
        options={
            "dry-run": BooleanOption("n", "dry-run"),
            "interactive": BooleanOption("i", "interactive"),
        },
    ),
    "stack": Command(),
    "stash": Command(),
    "filename": Option("f", "--file",
        short_description=u"write report to this file")
})

ModuleNotFoundError: No module named 'opts'

#### opts.add_md_help_argument(parser), opts.model_opts(parser), opts.train_opts(parser)
这几个方法没有找到，另外都用了argparse了，为什么还要用opts呢，他的好处体现在哪里？？？
好像误会了，这里的opts并不是网上的包，而是他们自己写的一个python文件。里面包含了他预设的几个参数，包括model_opts核train_opts

#### nn.Moudle.state_dict(destination=None, prefix='')
Returns a dictionary containing a whole state of the module.<br>
Both parameters and persistent buffers (e.g. running averages) are included. Keys are corresponding parameter and buffer names.<br>
model_state_dict = (model.module.state_dict() if len(opt.gpus) > 1 else model.state_dict())
#### torch.save(obj, f, pickle_module=$<$module 'pickle' from '/home/jenkins/miniconda/lib/python3.5/pickle.py'$>$, pickle_protocol=2)
Saves an object to a disk file.<br>
Parameters:	<br>
obj – saved object<br>
f – a file-like object (has to implement fileno that returns a file descriptor) or a string containing a file name<br>
pickle_module – module used for pickling metadata and objects<br>
pickle_protocol – can be specified to override the default protocol<br>

#### torch.load(f, map_location=None, pickle_module=$<$module 'pickle' from '/home/jenkins/miniconda/lib/python3.5/pickle.py'$>$)
Loads an object saved with torch.save() from a file.<br>
torch.load can dynamically remap storages to be loaded on a different device using the map_location argument. If it’s a callable, it will be called with two arguments: storage and location tag. It’s expected to either return a storage that’s been moved to a different location, or None (and the location will be resolved using the default method). If this argument is a dict it’s expected to be a mapping from location tags used in a file, to location tags of the current system.<br>
By default the location tags are ‘cpu’ for host tensors and ‘cuda:device_id’ (e.g. ‘cuda:2’) for cuda tensors. User extensions can register their own tagging and deserialization methods using register_package.<br>
Parameters:	<br>
f – a file-like object (has to implement fileno that returns a file descriptor, and must implement seek), or a string containing a file name<br>
map_location – a function or a dict specifying how to remap storage locations<br>
pickle_module – module used for unpickling metadata and objects (has to match the pickle_module used to serialize file)<br>

In [ ]:
>>> torch.load('tensors.pt')
# Load all tensors onto the CPU
>>> torch.load('tensors.pt', map_location=lambda storage, loc: storage)
# Map tensors from GPU 1 to GPU 0
>>> torch.load('tensors.pt', map_location={'cuda:1':'cuda:0'})

#### torch.manual_seed(seed)[source]
Sets the seed for generating random numbers. And returns a torch._C.Generator object.
#### cuda.set_device(opt.gpus[0])
Sets the current device.<br>
Usage of this function is discouraged in favor of device. In most cases it’s better to use CUDA_VISIBLE_DEVICES environmental variable.<br>
- torch.cuda

This package adds support for CUDA tensor types, that implement the same function as CPU tensors, but they utilize GPUs for computation.
It is lazily initialized, so you can always import it, and use is_available() to determine if your system supports CUDA.<br>

#### cc = CrayonClient(hostname=opt.log_server)
- get_experiment_names()

Returns a list of string containing the name of all the experiments on the server.<br>
- create_experiment(xp_name, zip_file=None)

Creates a new experiment with name xp_name and returns a CrayonExperiment object.
If zip_file is provided, this experiment is initialized with the content of the zip file (see CrayonExperiment.to_zip to get the zip file).

#### onmt.Loss.MemoryEfficientLoss
？？？？

#### opt.extra_shuffle and epoch > opt.curriculum:
opt.extra_shuffle也没找到这个变量。opt.curriculum不知道是干什么的？？？

#### torch.randperm(n, out=None) → LongTensor
Returns a random permutation of integers from 0 to n - 1.

#### opt.truncated_decoder if opt.truncated_decoder
Truncated BPTT is an approximation of full BPTT that is preferred for long sequences, since full BPTT’s forward/backward cost per parameter update becomes very high over many time steps. The downside is that the gradient can only flow back so far due to that truncation, so the network can’t learn dependencies that are as long as in full BPTT.

#### if dec_state is not None: dec_state.detach()
？？？？

#### model_state_dict = {k: v for k, v in model_state_dict.items() if 'generator' not in k}
为什么要去除含generator的？？？

#### os.path.abspath(path)
获得绝对路径
Return a normalized absolutized version of the pathname path. On most platforms, this is equivalent to calling the function normpath()

#### dataset = torch.load(opt.data) 333
不是只能用来读取参数的啊？？还是这里只是读了框架


#### class torch.nn.DataParallel(module, device_ids=None, output_device=None, dim=0)[source]
Implements data parallelism at the module level.<br>
This container parallelizes the application of the given module by splitting the input across the specified devices by chunking in the batch dimension. In the forward pass, the module is replicated on each device, and each replica handles a portion of the input. During the backwards pass, gradients from each replica are summed into the original module.<br>
The batch size should be larger than the number of GPUs used. It should also be an integer multiple of the number of GPUs so that each chunk is the same size (so that each GPU processes the same number of samples).<br>
Parameters:	<br>
module – module to be parallelized<br>
device_ids – CUDA devices (default: all devices)<br>
output_device – device location of output (default: device_ids[0])<br>

## onmt.Loss.py
#### nn.NLLLoss(weight, size_average=False)
class torch.nn.NLLLoss(weight=None, size_average=True, ignore_index=-100)[source]<br>
The negative log likelihood loss. It is useful to train a classification problem with n classes<br>
If provided, the optional argument weights should be a 1D Tensor assigning weight to each of the classes.<br>
This is particularly useful when you have an unbalanced training set.<br>
The input given through a forward call is expected to contain log-probabilities of each class: input has to be a 2D Tensor of size (minibatch, n)<br>
Obtaining log-probabilities in a neural network is easily achieved by adding a LogSoftmax layer in the last layer of your network.<br>
You may use CrossEntropyLoss instead, if you prefer not to add an extra layer.<br>
The target that this loss expects is a class index (0 to N-1, where N = number of classes)<br>



## CopyGenerator.py
### torch.nn.functional
包括：Convolution functions，Pooling functions，Non-linear activation functions，Normalization functions，Linear functions，Dropout functions，Distance functions，Loss functions，Vision functions
### torch.max(input, dim, keepdim=False, out=None) -> (Tensor, LongTensor)
Returns the maximum value of each row of the input Tensor in the given dimension dim. The second return value is the index location of each maximum value found (argmax).
### torch.sort(input, dim=None, descending=False, out=None) -> (Tensor, LongTensor)
Sorts the elements of the input Tensor along a given dimension in ascending order by value.<br>
If dim is not given, the last dimension of the input is chosen.<br>
If descending is True then the elements are sorted in descending order by value.<br>
A tuple of (sorted_tensor, sorted_indices) is returned, where the sorted_indices are the indices of the elements in the original input Tensor.<br>
### cpu()
Returns a CPU copy of this tensor if it’s not already on the CPU
### _, ids = dttn[0].cpu().data.sort(0, descending = True) L53
这个式子中的cpu不知道是为了做什么？？

## onmt.Optim.py
#### torch.optim
包含多个优化器，构建的时候使用要调整的变量及学习率等作为参数，调用时，使用step方法<br>
for input, target in dataset:<br>
&nbsp;&nbsp; optimizer.zero_grad()<br>
&nbsp;&nbsp; output = model(input)<br>
&nbsp;&nbsp; loss = loss_fn(output, target)<br>
&nbsp;&nbsp; loss.backward()<br>
&nbsp;&nbsp; optimizer.step()<br>
#### self.optimizer.param_groups[0]['lr'] = self.lr
optimizer里面没有param_groups啊？？？
#### torch.nn.utils.clip_grad_norm(parameters, max_norm, norm_type=2)[source]
Clips gradient norm of an iterable of parameters.<br>
The norm is computed over all gradients together, as if they were concatenated into a single vector. Gradients are modified in-place.<br>
Returns: Total norm of the parameters (viewed as a single vector).

## onmt.IO.py
#### torchtext.data, torchtext.vocab
木找到相关的内容，？？？？
#### from collections import Counter
我们都知道，Python拥有一些内置的数据类型，比如str, int, list, tuple, dict等， collections模块在这些内置数据类型的基础上，提供了几个额外的数据类型：<br>
- namedtuple(): 生成可以使用名字来访问元素内容的tuple子类
- deque: 双端队列，可以快速的从另外一侧追加和推出对象
- Counter: 计数器，主要用来计数
- OrderedDict: 有序字典
- defaultdict: 带有默认值的字典
- namedtuple()

namedtuple主要用来产生可以使用名称来访问元素的数据对象，通常用来增强代码的可读性， 在访问一些tuple类型的数据时尤其好用。
#### torchtext.vocab.Vocab(merged, specials = [PAD_WORD, BOS_WORD, EOS_WORD], max_size = vocab_size)
还是torchtext？？？

# 后面的大概的任务
- 得去看一下torchtext了， 汗一个
- 继续看openNMT的代码
- 看完后，总结画个草图。
- 回头把readPaper中的图片给加上
- positional embedding的问题还没有搞清楚
- stats = onmt.Loss.Statistics()
- onmt.Loss.MemoryEfficientLoss
- nn.Module的train mode和eval mode具体的区别在哪里？
- onmt.Loss.NMTCriterion